In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
# ~\.conda\envs\rag\python.exe

c:\Users\Agyeiwaa\.conda\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [1]:
from pinecone import Pinecone, ServerlessSpec
import os

# Initialize Pinecone with the API key
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create an index with the specified parameters
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [3]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'professor': 'Dr. Emily Carter',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Carter's lectures are engaging and clear. She has a deep understanding of the subject and makes difficult concepts easy to understand."},
 {'professor': 'Prof. John Smith',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Smith is very knowledgeable, but sometimes his explanations are a bit too fast. Overall, a good professor.'},
 {'professor': 'Dr. Linda Johnson',
  'subject': 'Chemistry',
  'stars': 3,
  'review': "Dr. Johnson's lectures are informative, but she can be hard to follow at times. The material is challenging, but she is approachable during office hours."},
 {'professor': 'Prof. Michael Brown',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Prof. Brown is amazing! His programming assignments are practical and really prepare you for the industry.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Biology',
  'stars': 2,
  'review': "Dr. Lee's classes are very co

In [16]:
from openai import OpenAI
import os
import json

data = json.load(open("reviews.json"))
api_key = ""

client = OpenAI(api_key=api_key)

processed_data = []
for review in data['reviews']:
    response=client.embeddings.create(
        input = review['review'],
        model ="text-embedding-ada-002",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
         "review":review["review"],
         "subject": review["subject"],
         "stars": review["stars"]
        } 
    })

In [17]:
processed_data[0]

{'values': [-0.013010872527956963,
  0.008578167296946049,
  0.012782382778823376,
  -0.026152310892939568,
  0.019741535186767578,
  0.0003892490058206022,
  0.0012395582161843777,
  -0.018122520297765732,
  -0.032197505235672,
  -0.01298475917428732,
  0.006205135025084019,
  0.03097018599510193,
  -0.004661195911467075,
  0.010438728146255016,
  0.015354527160525322,
  -0.0024644278455525637,
  0.026922648772597313,
  0.0023338620085269213,
  -0.011463669128715992,
  -0.020172402262687683,
  -0.011718272231519222,
  -0.0056632873602211475,
  -0.009302807040512562,
  0.006335700862109661,
  -0.01778304949402809,
  -0.015132565051317215,
  0.0224573016166687,
  -0.012815023772418499,
  -0.005646966863423586,
  -0.002505229553207755,
  0.01934983767569065,
  -0.007755603175610304,
  -0.020316023379564285,
  -0.04721255972981453,
  0.00778171606361866,
  -0.0054282695055007935,
  -0.013337287120521069,
  -0.018618669360876083,
  0.0028691813349723816,
  0.0016565524274483323,
  0.011000

In [22]:
from pinecone import Pinecone
import os

# Initialize Pinecone with the API key
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index('rag')
index.upsert(
    vectors =processed_data,
    namespace = "ns1"
)

c:\Users\Agyeiwaa\.conda\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


{'upserted_count': 20}

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}